# Importing the Required Libraries 

In [203]:
# loading the reqed libraries for scraping from Twitter and Reddit 
import tweepy # twitter api module 
import plotly.express as px 
import panel as pn
pn.extension('plotly')
import hvplot.pandas # plotting and visuliaizng the data 
import praw # reddit api module 
import os  # os module to access the .env file 
from dotenv import load_dotenv 
import torch # required library for transformers models 
import pandas as pd # data manipulation 

from bs4 import BeautifulSoup as bs # scraping and html parsing for some sites
import requests as rq # required for scraping from finviz.com

from transformers import AutoTokenizer, AutoModelForSequenceClassification # NLP 
import re

# Required to scrape finviz data for news articles and insider trades 
!pip install -U git+https://github.com/JQH84/finviz
import finviz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/JQH84/finviz to /private/var/folders/h4/m4qlztkx7ljd8rgkq9h9bwpr0000gn/T/pip-req-build-h2sg9r9m
  Running command git clone -q https://github.com/JQH84/finviz /private/var/folders/h4/m4qlztkx7ljd8rgkq9h9bwpr0000gn/T/pip-req-build-h2sg9r9m
  Resolved https://github.com/JQH84/finviz to commit 415b4946b83d4e9052c9725cd71fd8a054310b7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [2]:
# Loading the env varibles for the twitter and Reddit 

#load the function to work with env file
load_dotenv()

#create the twitter api access varibles 
twit_api= os.getenv("TWIT_API")
twit_secret = os.getenv("TWIT_SECRET")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# authentication for the twitter api 
auth = tweepy.OAuthHandler(twit_api, twit_secret)
auth.set_access_token(access_token, access_token_secret)

#createing the API object
t_api = tweepy.API(auth)

#create the reddit api access object and set to read-only 
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_SEC"),
    password=os.getenv("REDDIT_PASS"),
    user_agent="stock_dashboard by u/CryptoFoDayz",
    username=os.getenv("REDDIT_USER"),
)
reddit.read_only = True

# Grabbing the Twitter Data

In [64]:
nvd_30 = t_api.full('1month', 'NVD')


AttributeError: 'API' object has no attribute 'full'

In [46]:
df = pd.DataFrame(nvd_30)

In [32]:
# search tweets
keywords = 'NVDA'
limit=1000

tweets = tweepy.Cursor(t_api.search_tweets, q=keywords, count=1000, tweet_mode='extended').items(limit)

# create DataFrame
columns = ['User', 'Tweet' , 'Time']
data = []

for tweet in tweets:
    data.append([tweet.user.screen_name, tweet.full_text , tweet.user.created_at])

df = pd.DataFrame(data, columns=columns)

pattern = '(RT @)\w+:'
df['Tweet'] =df['Tweet'].apply(lambda x: re.sub(pattern , '' , x ) )

print(df)
print(df.dtypes)
df.to_csv('NVDA')
df.to_clipboard

              User                                              Tweet  \
0          AmakaCd  RT @delltronic2: @Scarcity_Mining @AmakaCd @no...   
1        ibrhm0172  RT @hakanrecepogluu: #Bitcoin \n\nArkadaşlar e...   
2        moon_bore  RT @kundunsan: In one week circulating supply ...   
3        TheRwopsa  RT @LATESTDAONEWS: Since we have invested in #...   
4          ts808hw  RT @JasonPLowery: At this rate, by the time I ...   
..             ...                                                ...   
995     OneCTrader                                @BTC_Archive #Space   
996     jiraphamiw                                    @BTC_Archive 😱😱   
997  mack_barnes0x  RT @CarpeNoctom: "There are certainly a large ...   
998       missuufe  RT @EShib_token: +10,000 watchlists in @CoinMa...   
999    CaniGarcia2  RT @Numbrs: Over the last year, the number of ...   

                         Time  
0   2022-02-09 13:56:57+00:00  
1   2021-01-28 12:36:44+00:00  
2   2021-10-07 17:40:33+00:

In [ ]:
panel.panel()

In [130]:
df.to_pickle("tweets.pkl")

In [99]:
pattern = '(RT @)\w+:'
df['Tweet'] =df['Tweet'].apply(lambda x: re.sub(pattern , '' ,x))
pattern = '(@)\w+'
df['Tweet'] =df['Tweet'].apply(lambda x: re.sub(pattern , '' ,x))


,User,Tweet,Time,sentiment
0,AmakaCd,In actuality the #ETH/BTC ratio and value has been trending up since 2020. ://t.…,2022-02-09 13:56:57+00:00,3
1,ibrhm0172,#Bitcoin \n\nArkadaşlar elinizde zararda olan coin varsa alış fiyatının ekran resmiyle birlikte bu tweet'in altına yorum…,2021-01-28 12:36:44+00:00,2
2,moon_bore,In one week circulating supply decreased 9.3B $MCC 🔥\n\nWhat’s next?\n\n-Multi Nodes with 1% daily passive income \n-Multi print…,2021-10-07 17:40:33+00:00,2
3,TheRwopsa,Since we have invested in #Looksrare 🚀 \n\nLet's spread the word about this awesome NFT Platform that Reward their traders…,2020-09-28 20:10:23+00:00,3
4,ts808hw,"At this rate, by the time I finish my thesis urging my superiors to consider the strategic importance of #BTC , US will h…",2017-01-05 21:21:08+00:00,2
...,...,...,...,...
995,OneCTrader,#Space,2021-02-26 07:31:33+00:00,2
996,jiraphamiw,😱😱,2021-08-31 14:09:30+00:00,2
997,mack_barnes0x,"""There are certainly a large number of coins that are commodities, including two of the biggest, which are $BTC and $ETH""…",2017-07-03 11:03:30+00:00,3
998,missuufe,"+10,000 watchlists in \n://t.co/BrrAV6ga6R\n#EShib #EuroShibaInu #EuroShiba #CMC #CoinMarketCap #BSC #BNB…",2021-08-22 10:56:48+00:00,2


In [204]:
#create the reddit api access object and set to read-only 
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_SEC"),
    password=os.getenv("REDDIT_PASS"),
    user_agent="stock_dashboard by u/CryptoFoDayz",
    username=os.getenv("REDDIT_USER"),
)
reddit.read_only = True

keyword = input('what keyword or stock symbol / crypto coin symbol would you like to analyze')
print('the following subreddits will be used for the sentiment analysis\n')

communities = []
posts = []

for subreddit in reddit.subreddits.search(keyword):
    if subreddit.display_name in 
    communities.append(subreddit.display_name)

for submission in reddit.subreddit(communities[1]).hot(limit=25):
    print(submission)
    



what keyword or stock symbol / crypto coin symbol would you like to analyze NIO


the following subreddits will be used for the sentiment analysis

spn8e0
sp3gnc
sppy32
spcpxa
spjl28
sp5uzg
spq9hm
spq5oe
soopua
spp9bp
spjx4p
spnrdr
sp48gb
sohzyl
spicdj
spgei3
spg5r8
spalb3
sogqup
sotqh4
soy55w
socthd
sov1ww
sp7ny2
so6lp0


In [169]:
reddit.subreddit('TSLA').

''

In [173]:
submission = reddit.subreddit('TSLA').hot()
submission

In [86]:
Keyword = reddit.subreddits.search('AAPL')

#comments = reddit.subreddit

{'q': 'AAPL', 'limit': 100}

In [120]:
crypto = rq.get('https://cryptopanic.com/api/v1/posts/?auth_token=f2d019ca021bcf8058683396c9719215a2ae9eea&filter=rising')
df_json = crypto.json()
dict_crypto = {}
for i in df_json['results']:
    dict_crypto = [df_json['results'][i]['title'] ,df_json['results'][i]['url'] ,df_json['results'][i]['created_at']]
crypto_news.head()

TypeError: list indices must be integers or slices, not dict

In [119]:
df_json['results'][0]['url']

'https://cryptopanic.com/news/14244022/Russia-Just-Made-Bitcoin-Legal'

In [93]:
import json 

df_json = crypto.json()
df_json['results'][0]['title']

titles = []
for title in range(len(df_json['results'])):
    titles.append(df_json['results'][title]['title'])
dict_titles ={'titles':titles}
df_titles = pd.DataFrame(dict_titles)
df_titles
df_titles['sentiment']=df_titles['titles'].apply(lambda x: sentiment_score(x[:]))
df_titles.groupby(by='sentiment').count()

,titles
0,Russia Just Made Bitcoin Legal!!
1,Microsoft Corp. Seeks For A Crypto Business Director To Build Its Web 3.0
2,Russia Moves Closer to Accepting Crypto as a Currency
3,El Salvador plans to issue first bitcoin bond next month
4,"JPMorgan’s Theoretical Bitcoin ($BTC) Price Target Sits at $150,000"
...,...
15,Bill Introduced To Let Tennessee Buy Bitcoin
16,Game review: Olympic Games Jam Beijing 2022
17,McDonald’s and Panera Bread file trademarks for virtual restaurants in the metaverse
18,"JPMorgan Says Bitcoin’s Fair Value Is $38K… But It Still Expects BTC To Eventually Hit $150,000"


In [94]:
df_titles['sentiment']=df_titles['titles'].apply(lambda x: sentiment_score(x[:]))
df_titles.groupby(by='sentiment').count()

,titles
sentiment,
1,1
2,18
3,1


# Perform Sentiment Analysis 

In [121]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# create tokenizer variable to chose the model 
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")



In [122]:
# encode the list of tweet to create the sentiment score from a function
def sentiment_score(tweet):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [123]:

df['sentiment'] = df['Tweet'].apply(lambda x: sentiment_score(x[:]))
df

,User,Tweet,Time,sentiment
0,AmakaCd,In actuality the #ETH/BTC ratio and value has been trending up since 2020. ://t.…,2022-02-09 13:56:57+00:00,3
1,ibrhm0172,#Bitcoin \n\nArkadaşlar elinizde zararda olan coin varsa alış fiyatının ekran resmiyle birlikte bu tweet'in altına yorum…,2021-01-28 12:36:44+00:00,2
2,moon_bore,In one week circulating supply decreased 9.3B $MCC 🔥\n\nWhat’s next?\n\n-Multi Nodes with 1% daily passive income \n-Multi print…,2021-10-07 17:40:33+00:00,2
3,TheRwopsa,Since we have invested in #Looksrare 🚀 \n\nLet's spread the word about this awesome NFT Platform that Reward their traders…,2020-09-28 20:10:23+00:00,3
4,ts808hw,"At this rate, by the time I finish my thesis urging my superiors to consider the strategic importance of #BTC , US will h…",2017-01-05 21:21:08+00:00,2
...,...,...,...,...
995,OneCTrader,#Space,2021-02-26 07:31:33+00:00,2
996,jiraphamiw,😱😱,2021-08-31 14:09:30+00:00,2
997,mack_barnes0x,"""There are certainly a large number of coins that are commodities, including two of the biggest, which are $BTC and $ETH""…",2017-07-03 11:03:30+00:00,3
998,missuufe,"+10,000 watchlists in \n://t.co/BrrAV6ga6R\n#EShib #EuroShibaInu #EuroShiba #CMC #CoinMarketCap #BSC #BNB…",2021-08-22 10:56:48+00:00,2


In [36]:
df

,User,Tweet,Time,sentiment
0,AmakaCd,RT @delltronic2: @Scarcity_Mining @AmakaCd @novogratz In actuality the #ETH/BTC ratio and value has been trending up since 2020. https://t.…,2022-02-09 13:56:57+00:00,3
1,ibrhm0172,RT @hakanrecepogluu: #Bitcoin \n\nArkadaşlar elinizde zararda olan coin varsa alış fiyatının ekran resmiyle birlikte bu tweet'in altına yorum…,2021-01-28 12:36:44+00:00,2
2,moon_bore,RT @kundunsan: In one week circulating supply decreased 9.3B $MCC 🔥\n\nWhat’s next?\n\n-Multi Nodes with 1% daily passive income \n-Multi print…,2021-10-07 17:40:33+00:00,2
3,TheRwopsa,RT @LATESTDAONEWS: Since we have invested in #Looksrare 🚀 \n\nLet's spread the word about this awesome NFT Platform that Reward their traders…,2020-09-28 20:10:23+00:00,3
4,ts808hw,"RT @JasonPLowery: At this rate, by the time I finish my thesis urging my superiors to consider the strategic importance of #BTC , US will h…",2017-01-05 21:21:08+00:00,2
...,...,...,...,...
995,OneCTrader,@BTC_Archive #Space,2021-02-26 07:31:33+00:00,2
996,jiraphamiw,@BTC_Archive 😱😱,2021-08-31 14:09:30+00:00,2
997,mack_barnes0x,"RT @CarpeNoctom: ""There are certainly a large number of coins that are commodities, including two of the biggest, which are $BTC and $ETH""…",2017-07-03 11:03:30+00:00,3
998,missuufe,"RT @EShib_token: +10,000 watchlists in @CoinMarketCap\nhttps://t.co/BrrAV6ga6R\n#EShib #EuroShibaInu #EuroShiba #CMC #CoinMarketCap #BSC #BNB…",2021-08-22 10:56:48+00:00,2


In [126]:
df_grouped= df.groupby(by ='year')
df_grouped.reset_index()
df_grouped.drop(columns ='User' , inplace = True)

In [127]:
df_grouped.hvplot.bar()

:Bars   [sentiment,Variable]   (value)

In [64]:
df_grouped.head()

,Tweet,Time
sentiment,,
1,42,42
2,720,720
3,238,238


In [174]:
symbols = ['NVDA','NIO','BITF']
urls=[]
for item in range(len(symbols)):
    urls.append(f"https://finviz.com/quote.ashx?t={symbols[item]}")

# Create a function to scrape from a given URL 
def finscrape(url):
    response = rq.get(url)
    
    soup = Beautifulsoup(response , 'html.parser')
    
    articles = soup.find_all

In [183]:
url ='https://finviz.com/quote.ashx?t=NVDA'
response = rq.get(url)

soup = bs(response.text , 'html.parser')

print(soup.prettify())

<!DOCTYPE HTML>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="noindex, nofollow" name="robots"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <title>
   Just a moment...
  </title>
  <style type="text/css">
   html, body {width: 100%; height: 100%; margin: 0; padding: 0;}
    body {background-color: #ffffff; color: #000000; font-family:-apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", Roboto, Oxygen, Ubuntu, "Helvetica Neue",Arial, sans-serif; font-size: 16px; line-height: 1.7em;-webkit-font-smoothing: antialiased;}
    h1 { text-align: center; font-weight:700; margin: 16px 0; font-size: 32px; color:#000000; line-height: 1.25;}
    p {font-size: 20px; font-weight: 400; margin: 8px 0;}
    p, .attribution, {text-align: center;}
    #spinner {margin: 0 auto 30px auto; display: block;}
    .att

In [185]:
!pip install -U git+https://github.com/JQH84/finviz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Cloning https://github.com/JQH84/finviz to /private/var/folders/h4/m4qlztkx7ljd8rgkq9h9bwpr0000gn/T/pip-req-build-yxv2wh7s
  Running command git clone -q https://github.com/JQH84/finviz /private/var/folders/h4/m4qlztkx7ljd8rgkq9h9bwpr0000gn/T/pip-req-build-yxv2wh7s
  Resolved https://github.com/JQH84/finviz to commit 919d00d0770566e3833d412b25067a50797e3c3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached tenacity-7.0.0-py2.py3-none-any.whl (23 kB)
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
  Using cached user_agent-0.1.10.tar.gz (20 kB)
  Created wheel for f

In [186]:
import finviz

In [199]:
news = finviz.get_news('NVDA')

news[0][1]

'How Nvidia Makes Money: Gaming, Data Center, and AI Platforms'

In [202]:
test = finviz.get_insider('nvda')
test

[{'Insider Trading': 'HUANG JEN HSUN',
  'Relationship': 'President and CEO',
  'Date': 'Jan 03',
  'Transaction': 'Option Exercise',
  'Cost': '3.62',
  '#Shares': '440,000',
  'Value ($)': '1,590,600',
  '#Shares Total': '5,618,925',
  'SEC Form 4': 'Jan 05 05:25 PM'},
 {'Insider Trading': 'HUANG JEN HSUN',
  'Relationship': 'President and CEO',
  'Date': 'Jan 03',
  'Transaction': 'Sale',
  'Cost': '302.17',
  '#Shares': '440,000',
  'Value ($)': '132,954,031',
  '#Shares Total': '5,178,925',
  'SEC Form 4': 'Jan 05 05:25 PM'},
 {'Insider Trading': 'Dabiri John',
  'Relationship': 'Director',
  'Date': 'Dec 23',
  'Transaction': 'Sale',
  'Cost': '297.50',
  '#Shares': '281',
  'Value ($)': '83,598',
  '#Shares Total': '4,912',
  'SEC Form 4': 'Dec 23 07:28 PM'},
 {'Insider Trading': 'JONES HARVEY C',
  'Relationship': 'Director',
  'Date': 'Dec 16',
  'Transaction': 'Sale',
  'Cost': '311.50',
  '#Shares': '41,484',
  'Value ($)': '12,922,266',
  '#Shares Total': '866,396',
  'SEC 